In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.tasks import Task

In [2]:
np.random.seed(42)

In [3]:
data = pd.read_csv("/opt/spark_data/HIGGS.csv", header=None, prefix = 'Column_', nrows=1_000_000)

In [4]:
len(data)

1000000

In [5]:
train, test = train_test_split(data, test_size=200_000, random_state=42)

In [6]:
print(len(train))
print(len(test))

800000
200000


In [7]:
test.head()

,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,...,Column_19,Column_20,Column_21,Column_22,Column_23,Column_24,Column_25,Column_26,Column_27,Column_28
987231,0.0,1.205664,-1.169791,0.015953,0.591559,-1.154139,0.452631,-0.141628,0.914326,0.000000,...,0.495069,0.348878,3.101961,0.915096,0.791421,1.016853,0.877905,0.922745,0.852537,0.777144
79954,1.0,0.937373,-1.523341,1.383686,1.053063,0.554343,2.004091,-2.284486,-1.723497,0.000000,...,-1.286340,0.040347,0.000000,0.483557,0.750305,0.988924,0.494695,0.735078,0.806357,0.675272
567130,1.0,0.850992,-1.590545,-1.461748,0.558573,0.545128,1.099378,-1.043728,-0.588137,2.173076,...,-1.666940,0.699583,0.000000,0.536244,1.222684,0.985939,1.143061,1.932193,1.225894,1.000200
500891,0.0,0.464660,0.297006,0.595228,2.033882,-0.480508,0.470037,0.247532,-0.136873,1.086538,...,-0.349414,-1.248819,0.000000,1.277576,0.976491,1.109844,1.008831,0.950556,0.775099,0.778901
55399,1.0,0.860875,0.421674,-1.339679,0.655404,1.008641,0.560545,0.974361,1.180980,0.000000,...,-1.600314,0.693479,0.000000,0.924148,1.059373,0.990987,0.886394,0.663309,0.879856,0.764028


In [ ]:
train.columns

In [8]:
roles = {
    "target": "Column_0"
}

In [9]:
task = Task(
    "binary",
)

In [10]:
automl = TabularAutoML(
    task=task,
    timeout=6000,
    general_params={
        "use_algos": [["lgb"]]
    },
    nested_cv_params={"cv": 2, "n_folds": None},
)

In [11]:
oof_pred = automl.fit_predict(train, roles=roles)

tdf ctr
<class 'lightautoml.transformers.base.UnionTransformer'>
<class 'lightautoml.transformers.base.UnionTransformer'>


In [12]:
test_pred = automl.predict(test)

In [13]:
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

In [14]:
print(f"OOF score: {roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])}")
print(f"TEST score: {roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])}")

OOF score: 0.8314776225531073
TEST score: 0.83619988900382
